In [3]:
#Template for Simple coding to Understand how to access json file
import pandas as pd
import requests
import json

# Fetch the JSON data from the URL
url = 'URL Link'
response = requests.get(url)
json_data = response.json()

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(json_data)
# Display the DataFrame
print(df)

In [32]:
#Access the ASANA and Import the data in json format
import pandas as pd
import requests
import json

ASANA_BASE_TASK_URL = "https://app.asana.com/api/1.0/tasks"
#Access the link for the ASANA: https://app.asana.com/0/1207638669590015/1207638670232784
PROJECT_ID = "1207638669590015" # This is hardcoded for the purpose of this  example, but I would recommend grabbing the ID you want from an additional API call
personal_access_token = "2/1195781365485212/1207621405062976:08cb575f0db7c127e6f3a4457568453b" # Grab this from env variables or secrets where you saved it!

headers = {
   "Authorization": f"Bearer {personal_access_token}"}

url = f"{ASANA_BASE_TASK_URL}?project={PROJECT_ID}"
# This constructed URL should look something like this: https://app.asana.com/api/1.0/tasks?project=1202095602265837


# Make the API Request. Note the .get() function, which corresponds to the HTTP GET method of our request
response = requests.get(url, headers=headers)


# If the status code is 200, it means the request was successful
if response.status_code == 200:
   tasks = response.json()
   print("The Access to the API was successful")
   print(tasks)
   # After this -- you can do whatever you want with the tasks that is useful to you!


# If status code is not 200, print an error message since something went wrong
else:
   print(f"Failed to retrieve tasks. Status code: {response.status_code}")
   

The Access to the API was successful
{'data': [{'gid': '1207638669590020', 'name': 'Scoping', 'resource_type': 'task', 'resource_subtype': 'default_task'}, {'gid': '1207638670232784', 'name': 'E2E Project Management', 'resource_type': 'task', 'resource_subtype': 'default_task'}]}


In [33]:
#Inspect the data
json_data = tasks
json_data

{'data': [{'gid': '1207638669590020',
   'name': 'Scoping',
   'resource_type': 'task',
   'resource_subtype': 'default_task'},
  {'gid': '1207638670232784',
   'name': 'E2E Project Management',
   'resource_type': 'task',
   'resource_subtype': 'default_task'}]}

In [34]:
#The data contains additional keyword namely 'data' in the beginning. Let's first remove it
json_data_cleaned = json_data['data']
json_data_cleaned

[{'gid': '1207638669590020',
  'name': 'Scoping',
  'resource_type': 'task',
  'resource_subtype': 'default_task'},
 {'gid': '1207638670232784',
  'name': 'E2E Project Management',
  'resource_type': 'task',
  'resource_subtype': 'default_task'}]

In [35]:
#Convert the json data into a DataFrame
df = pd.DataFrame(json_data_cleaned)

df

,gid,name,resource_type,resource_subtype
0,1207638669590020,Scoping,task,default_task
1,1207638670232784,E2E Project Management,task,default_task


************************************************
### Join the Project Name with Custom Fields ###
************************************************

In [141]:
####Code to Access the Config Fields along with Main ASANA fields#####

#Access the ASANA and Import the data in json format
import pandas as pd
import requests
import json

ASANA_BASE_TASK_URL = "https://app.asana.com/api/1.0/tasks"
#Access the link for the ASANA: https://app.asana.com/0/1207638669590015/1207638670232784
PROJECT_ID = "1207638669590015" # This is hardcoded for the purpose of this  example, but I would recommend grabbing the ID you want from an additional API call
#PROJECT_ID = "1206713753906403"
personal_access_token = "2/1195781365485212/1207621405062976:08cb575f0db7c127e6f3a4457568453b" # Grab this from env variables or secrets where you saved it!

headers = {
   "Authorization": f"Bearer {personal_access_token}"}

url = f"{ASANA_BASE_TASK_URL}?project={PROJECT_ID}&opt_fields=name,custom_fields.name,custom_fields.display_value"
# This constructed URL should look something like this: https://app.asana.com/api/1.0/tasks?project=1202095602265837


# Make the API Request. Note the .get() function, which corresponds to the HTTP GET method of our request
response = requests.get(url, headers=headers)


# If the status code is 200, it means the request was successful
if response.status_code == 200:
   tasks = response.json()
   print("The Access to the API was successful")
   print(tasks)
   # After this -- you can do whatever you want with the tasks that is useful to you!


# If status code is not 200, print an error message since something went wrong
else:
   print(f"Failed to retrieve tasks. Status code: {response.status_code}")
   

The Access to the API was successful
{'data': [{'gid': '1207638669590020', 'custom_fields': [{'gid': '1207638669590025', 'name': 'NWP Product', 'display_value': 'Proton'}], 'name': 'Scoping'}, {'gid': '1207638670232784', 'custom_fields': [{'gid': '1207638669590025', 'name': 'NWP Product', 'display_value': 'Proton, Prime-Times'}], 'name': 'E2E Project Management'}]}


In [142]:
#Inspect the data
json_data = tasks
json_data

#The data contains additional keyword namely 'data' in the beginning. Let's first remove it
json_data_cleaned = json_data['data']
json_data_cleaned

[{'gid': '1207638669590020',
  'custom_fields': [{'gid': '1207638669590025',
    'name': 'NWP Product',
    'display_value': 'Proton'}],
  'name': 'Scoping'},
 {'gid': '1207638670232784',
  'custom_fields': [{'gid': '1207638669590025',
    'name': 'NWP Product',
    'display_value': 'Proton, Prime-Times'}],
  'name': 'E2E Project Management'}]

In [143]:
# Just extract the Custome Field. Flatten the 'custom_fields' columns
json_data_normalized = pd.json_normalize(json_data_cleaned, record_path=['custom_fields'])
json_data_normalized

,gid,name,display_value
0,1207638669590025,NWP Product,Proton
1,1207638669590025,NWP Product,"Proton, Prime-Times"


In [145]:
#Extract Intake Title
intake_title_data = pd.json_normalize(json_data_cleaned)[['name']].rename(columns={'name':'Intake_Title'})

In [146]:
#Combine the Intake Title dataframe and Custom Field dataframe
intake_data = pd.concat([intake_title_data, json_data_normalized], axis=1)

In [147]:
intake_data.head(2)

,Intake_Title,gid,name,display_value
0,Scoping,1207638669590025,NWP Product,Proton
1,E2E Project Management,1207638669590025,NWP Product,"Proton, Prime-Times"


In [148]:
intake_data.to_csv('/Users/zishan/Documents/asana_intake_csv_file.csv', index=False)

***
### Access of the **NMT Folder** ###
***

In [ ]:
####Experiment to collect Network Planning Fields in ASANA#####

#Access the ASANA and Import the data in json format
import pandas as pd
import requests
import json

ASANA_BASE_TASK_URL = "https://app.asana.com/api/1.0/tasks"
#Access the link for the ASANA: https://app.asana.com/0/1207638669590015/1207638670232784
#PROJECT_ID = "1207638669590015" # This is hardcoded for the purpose of this  example, but I would recommend grabbing the ID you want from an additional API call
PROJECT_ID = "1206713753906403"
personal_access_token = "2/1195781365485212/1207621405062976:08cb575f0db7c127e6f3a4457568453b" # Grab this from env variables or secrets where you saved it!

headers = {
   "Authorization": f"Bearer {personal_access_token}"}

url = f"{ASANA_BASE_TASK_URL}?project={PROJECT_ID}&opt_fields=name,permalink_url,custom_fields.name,custom_fields.display_value"
# This constructed URL should look something like this: https://app.asana.com/api/1.0/tasks?project=1202095602265837


# Make the API Request. Note the .get() function, which corresponds to the HTTP GET method of our request
response = requests.get(url, headers=headers)


# If the status code is 200, it means the request was successful
if response.status_code == 200:
   tasks = response.json()
   print("The Access to the API was successful")
   #print(tasks)
   # After this -- you can do whatever you want with the tasks that is useful to you!


# If status code is not 200, print an error message since something went wrong
else:
   print(f"Failed to retrieve tasks. Status code: {response.status_code}")
   

In [34]:
#Inspect the JSON extracted data
json_data = tasks
#Print just 500 character of Json data
print(json.dumps(json_data, indent=1, sort_keys=True)[:500])

{
 "data": [
  {
   "custom_fields": [
    {
     "display_value": "Timoth\u00e9e Cuignet",
     "gid": "1207008328334398",
     "name": "NMT UT Owner"
    },
    {
     "display_value": "ST Scheduling",
     "gid": "1206982449331430",
     "name": "NMT_Tech_Umbrella_Topics_OP25"
    },
    {
     "display_value": "ST Tech - MMPO",
     "gid": "1206328848168146",
     "name": "Lead_Tech/Sci_team_OP25"
    },
    {
     "display_value": "TBD",
     "gid": "1206854117206259",
     "name": "OP Stat


In [35]:
#The JSON data contains additional keyword namely 'data' in the beginning. Let's first remove it
json_data_cleaned = json_data['data']
#Print just 500 character of Json data
print(json.dumps(json_data_cleaned, indent=1, sort_keys=True)[:500])

[
 {
  "custom_fields": [
   {
    "display_value": "Timoth\u00e9e Cuignet",
    "gid": "1207008328334398",
    "name": "NMT UT Owner"
   },
   {
    "display_value": "ST Scheduling",
    "gid": "1206982449331430",
    "name": "NMT_Tech_Umbrella_Topics_OP25"
   },
   {
    "display_value": "ST Tech - MMPO",
    "gid": "1206328848168146",
    "name": "Lead_Tech/Sci_team_OP25"
   },
   {
    "display_value": "TBD",
    "gid": "1206854117206259",
    "name": "OP Status - NMT Tech"
   },
   {
    "d


In [36]:
# First of all extract all the Custome Fields from JSON file. Flatten the 'custom_fields' columns
json_data_normalized = pd.json_normalize(json_data_cleaned, record_path=['custom_fields'])
json_data_normalized.head(5)

,gid,name,display_value
0,1207008328334398,NMT UT Owner,Timothée Cuignet
1,1206982449331430,NMT_Tech_Umbrella_Topics_OP25,ST Scheduling
2,1206328848168146,Lead_Tech/Sci_team_OP25,ST Tech - MMPO
3,1206854117206259,OP Status - NMT Tech,TBD
4,1206328690471503,Requestor Login_OP25,saucsing


In [40]:


unique_gid = json_data_normalized['gid'].unique()
unique_gid
custom_field_data = pd.DataFrame()
for gid_value in unique_gid:
    #Extract the data for each gid and pivot it to get rows as column
    pivoted_data = json_data_normalized[json_data_normalized['gid'] == gid_value].pivot(columns="name", values='display_value')
    #print(pivoted_data)
    ##Start preparing the data to combine all the Data togethetr for each GID
    #Reset the index of the Pivoted data, so merge can happen on the reset index; to avoid extra rows creations
    pivoted_data = pivoted_data.reset_index()
    #Drop the extra Index column that resulted due to reseting the index
    pivoted_data = pivoted_data.drop(pivoted_data.columns[0], axis=1)
    ##Combine all the pivoted data for each gid
    custom_field_data = pd.concat([custom_field_data, pivoted_data], axis=1)
    
custom_field_data.head(5)


name,NMT UT Owner,NMT_Tech_Umbrella_Topics_OP25,Lead_Tech/Sci_team_OP25,OP Status - NMT Tech,Requestor Login_OP25,ATS_Finance_BCT_Validation_OP25,Total_Entitlement_OP25,ATS_Business_Umbrella_Topic(s)_OP25,ATS_Safety_criticality_OP25,ATS_Compliance_criticality_OP25,...,TFS_Categories_OP25,TFS_stakeholder_OP25,Impacted_Country_OP25,TFS_Mode_OP25,HC_Decision_TFS_OP25,Tech Scoping Status - TFS,Gatekeeper Status,RSS Product Manager,RSS Product Team,RFT Funding Available?
0,Timothée Cuignet,ST Scheduling,ST Tech - MMPO,TBD,saucsing,BCT Approved,46700000,"MMN Labor Planning, ST Ops Planning & Scheduling",P3 - Not Critical,P3 - Not Critical,...,Accounting Automation,"Finance , Technology Teams (MMPT, TCDA, LMP, e...","NA, EU, IN, JP","Middle Mile, Last Mile, Air, Sort Center, Deli...",TBD,Ready for GTT Review,PMO review,Jayesh Poobathy,Performance,Yes
1,"Noor Adnan, Alex Chiriac, Timothée Cuignet",ST Scheduling,ST Tech - MMPO,TBD,clued,No financial benefit,0,ST Ops Planning & Scheduling,P3 - Not Critical,P3 - Not Critical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Souvik Halder,Sustainability,Yes
2,Alex Chiriac,1P Partnership,NMT Tech (SnT & DnA),TBD,ssawaya,BCT Unvalidated,52400000,"Country Expansion, Scaling & Automation",P3 - Not Critical,P3 - Not Critical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Souvik Halder,Sustainability,NaN
3,"Marina Rinaldi, Nandni Gaur",Network Planning,NMT Tech (SnT & DnA),TBD,nandgaur,No financial benefit,0,Network Planning,P3 - Not Critical,P3 - Not Critical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alex Chiriac,1P Partnership,NMT Tech (SnT & DnA),TBD,chirlex,No financial benefit,0,"Amazon Freight, Intermodal, On-Road Operator E...",P3 - Not Critical,P3 - Not Critical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#Read the title of the tasks
intake_title_data = pd.json_normalize(json_data_cleaned)[['name','permalink_url']].rename(columns={'name':'Request Heading', 'permalink_url':'Request_Link'})
intake_title_data.head(5)

,Request Heading,Request_Link
0,Integration of M3 volume flows (except SWA) in...,https://app.asana.com/0/1206713753906403/12073...
1,Automated Commercial Carrier Feedback Processing,https://app.asana.com/0/1206713753906403/12073...
2,TheyPay2ShipCon Cost Prediction Model,https://app.asana.com/0/1206713753906403/12073...
3,[Late Request] 1Click: Prime-Times rearchitect...,https://app.asana.com/0/1206713753906403/12075...
4,ST Data Engineering Efforts,https://app.asana.com/0/1206713753906403/12073...


In [43]:
#Combine the Intake Title dataframe and Custom Field dataframe
intake_data = pd.concat([intake_title_data, custom_field_data], axis=1)
intake_data.head(2)

,Request Heading,Request_Link,NMT UT Owner,NMT_Tech_Umbrella_Topics_OP25,Lead_Tech/Sci_team_OP25,OP Status - NMT Tech,Requestor Login_OP25,ATS_Finance_BCT_Validation_OP25,Total_Entitlement_OP25,ATS_Business_Umbrella_Topic(s)_OP25,...,TFS_Categories_OP25,TFS_stakeholder_OP25,Impacted_Country_OP25,TFS_Mode_OP25,HC_Decision_TFS_OP25,Tech Scoping Status - TFS,Gatekeeper Status,RSS Product Manager,RSS Product Team,RFT Funding Available?
0,Integration of M3 volume flows (except SWA) in...,https://app.asana.com/0/1206713753906403/12073...,Timothée Cuignet,ST Scheduling,ST Tech - MMPO,TBD,saucsing,BCT Approved,46700000,"MMN Labor Planning, ST Ops Planning & Scheduling",...,Accounting Automation,"Finance , Technology Teams (MMPT, TCDA, LMP, e...","NA, EU, IN, JP","Middle Mile, Last Mile, Air, Sort Center, Deli...",TBD,Ready for GTT Review,PMO review,Jayesh Poobathy,Performance,Yes
1,Automated Commercial Carrier Feedback Processing,https://app.asana.com/0/1206713753906403/12073...,"Noor Adnan, Alex Chiriac, Timothée Cuignet",ST Scheduling,ST Tech - MMPO,TBD,clued,No financial benefit,0,ST Ops Planning & Scheduling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Souvik Halder,Sustainability,Yes


In [44]:
intake_data.to_csv('/Users/zishan/Documents/asana_intake_csv_file.csv', index=False)

intake_data.to_excel('/Users/zishan/Documents/asana_intake_excel_file.xlsx', index=False, sheet_name='Sheet1')